In [54]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()
driver.get("https://scholar.google.com/schhp?hl=en&as_sdt=0,48")

search = driver.find_element(By.NAME, 'q')
search.send_keys("bats")
search.send_keys(Keys.RETURN)

url = driver.current_url
titles = []
links = []
authors = []
description = []

response=requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')
for item in soup.select('[data-lid]'):
    t = item.select('h3')[0].get_text()
    titles.append(t)
    l = item.select('a')[0]['href']
    links.append(l)
    a = item.select('.gs_a')[0].get_text()
    authors.append(a)
    d = item.select('.gs_rs')[0].get_text()
    description.append(d)
    
    
driver.quit()

df = pd.DataFrame(list(zip(titles, links, authors, description)), columns = ["Titles", "Links", "Authors", "Description"])
print(df)

                                    Titles  \
5  [BOOK][B] Bats: a community perspective   

                                               Links  \
5  https://books.google.com/books?hl=en&lr=&id=cr...   

                                Authors  \
5  JS Findley - 1995 - books.google.com   

                                         Description  
5  … This book begins with an overview of bat bio...  
